In [1]:
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA, KernelPCA
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.feature_selection import VarianceThreshold, RFE, SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier, AdaBoostClassifier

In [3]:
data = pd.read_csv('data.csv')
print data.head()

         action_type combined_shot_type  game_event_id   game_id      lat  \
0          Jump Shot          Jump Shot             10  20000012  33.9723   
1          Jump Shot          Jump Shot             12  20000012  34.0443   
2          Jump Shot          Jump Shot             35  20000012  33.9093   
3          Jump Shot          Jump Shot             43  20000012  33.8693   
4  Driving Dunk Shot               Dunk            155  20000012  34.0443   

   loc_x  loc_y       lon  minutes_remaining  period   ...          shot_type  \
0    167     72 -118.1028                 10       1   ...     2PT Field Goal   
1   -157      0 -118.4268                 10       1   ...     2PT Field Goal   
2   -101    135 -118.3708                  7       1   ...     2PT Field Goal   
3    138    175 -118.1318                  6       1   ...     2PT Field Goal   
4      0      0 -118.2698                  6       2   ...     2PT Field Goal   

          shot_zone_area  shot_zone_basic  shot_zo

# Load the Data 1

In [23]:
data = pd.read_csv('data.csv')

data.set_index('shot_id', inplace=True)
data.drop(['game_event_id','playoffs', 'game_id', 'lat', 'lon','season', 'period', 'team_id', 'team_name','game_date'], axis=1, inplace=True)
actiontypes = dict(data.action_type.value_counts())

data['type'] = data.apply(lambda row: row['action_type'] if actiontypes[row['action_type']] > 20\
 #                         else row['combined_shot_type'], axis=1)

action_map = {action: i for i, action in enumerate(data['action_type'].unique())}
data['action_type_enumerated'] = data['action_type'].map(action_map)
ct = pd.crosstab(data.shot_made_flag, data.action_type_enumerated).apply(lambda x:x/x.sum(), axis=0)
temp = list(zip(ct.values[1, :], ct.columns))
temp.sort()
new_map = {}
for index, (acc, old_number) in enumerate(temp):
    new_map[old_number] = index
data['action_type_new_enumeration'] = data.action_type_enumerated.map(new_map)

data.drop(['action_type', 'combined_shot_type','action_type_enumerated'], axis=1, inplace=True)

data['away'] = data.matchup.str.contains('@')
data.drop('matchup', axis=1, inplace=True)

data['distance'] = data.apply(lambda row: row['shot_distance'] if row['shot_distance'] <45 else 45, axis=1)

data['time_remaining'] = data.apply(lambda row: row['minutes_remaining'] * 60 + row['seconds_remaining'], axis=1)
data['last_moments'] = data.apply(lambda row: 1 if row['time_remaining'] < 3 else 0, axis=1)


#data = pd.get_dummies(data['type'],prefix="action_type")

data.drop(['minutes_remaining','time_remaining','seconds_remaining','away', "opponent",'distance'], axis=1, inplace=True)


categorial_cols =[ "shot_type", "shot_zone_area", "shot_zone_basic",
           "shot_zone_range"]

data_cl = data.copy() # create a copy of data frame
target = data_cl['shot_made_flag'].copy()
data_cl.drop('shot_made_flag', axis=1, inplace=True)


for cc in categorial_cols:
    dummies = pd.get_dummies(data_cl[cc])
    dummies = dummies.add_prefix("{}#".format(cc))
    data_cl.drop(cc, axis=1, inplace=True)
    data_cl = data_cl.join(dummies)
    
data_cl.head()




from sklearn.cross_validation import train_test_split

mask = data['shot_made_flag'].isnull()



# Separate dataset for TRAIN and VALIDATION
X = data_cl[~mask]
Y = data.shot_made_flag[~mask] 

X, X_val, Y, Y_val = train_test_split(X,Y, test_size=0.1)

# TEST
data_submit = data_cl[mask]



SyntaxError: invalid syntax (<ipython-input-23-312705d65850>, line 9)

In [22]:
X.head()

,loc_x,loc_y,distance,last_moments,shot_zone_area#Back Court(BC),shot_zone_area#Center(C),shot_zone_area#Left Side Center(LC),shot_zone_area#Left Side(L),shot_zone_area#Right Side Center(RC),shot_zone_area#Right Side(R),...,type#Turnaround Fadeaway shot,type#Turnaround Jump Shot,shot_zone_range#16-24 ft.,shot_zone_range#24+ ft.,shot_zone_range#8-16 ft.,shot_zone_range#Back Court Shot,shot_zone_range#Less Than 8 ft.,shot_type#2PT Field Goal,shot_type#3PT Field Goal,shotLocationCluster
1122,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1
7909,-18,41,4,0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,11
3127,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1
26160,-109,218,24,0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,8
11950,-13,41,4,0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,11


# Load the data 2

In [12]:
import numpy as np
import pandas as pd

from sklearn.cross_validation import cross_val_score

# Load data and roughly clean it, then sort as game date
df = pd.read_csv("data.csv")
#df.set_index('shot_id', inplace=True)

df.drop(['game_event_id','playoffs','period', 'game_id', 'lat', 'lon', 'team_id', 'team_name','season'], axis=1, inplace=True)
mask = df['shot_made_flag'].isnull()

# Clean data
actiontypes = dict(df.action_type.value_counts())
df['type'] = df.apply(lambda row: row['action_type'] if actiontypes[row['action_type']] > 20\
                          else row['combined_shot_type'], axis=1)
df.drop(['action_type','game_date','opponent', 'combined_shot_type'], axis=1, inplace=True)


# POLAR CORDINATES
#df['dist'] = np.sqrt(df['loc_x']**2 + df['loc_y']**2)

#loc_x_zero = df['loc_x'] == 0
#df['angle'] = np.array([0]*len(df))
#df['angle'][~loc_x_zero] = np.arctan(df['loc_y'][~loc_x_zero] / df['loc_x'][~loc_x_zero])
#df['angle'][loc_x_zero] = np.pi / 2 


#df['away'] = df.matchup.str.contains('@')
df.drop('matchup', axis=1, inplace=True)

df['distance'] = df.apply(lambda row: row['shot_distance'] if row['shot_distance'] <45 else 45, axis=1)

df['time_remaining'] = df.apply(lambda row: row['minutes_remaining'] * 60 + row['seconds_remaining'], axis=1)
df['last_moments'] = df.apply(lambda row: 1 if row['time_remaining'] < 3 else 0, axis=1)

df.drop(['minutes_remaining','time_remaining','shot_distance','seconds_remaining'], axis=1, inplace=True)

data = df.copy()





In [13]:

categorial_cols=["shot_zone_area", "shot_zone_basic",'type',
           "shot_zone_range",'shot_type']

for cc in categorial_cols:
    dummies = pd.get_dummies(data[cc])
    dummies = dummies.add_prefix("{}#".format(cc))
    data.drop(cc, axis=1, inplace=True)
    data = data.join(dummies)
    
from sklearn.cross_validation import train_test_split




# Separate dataset for TRAIN and VALIDATION
X = data[~mask]
Y = df.shot_made_flag[~mask] 

X, X_val, Y, Y_val = train_test_split(X,Y, test_size=0.1)

# TEST
data_submit = data[mask]
shot_id = df[mask]["shot_id"]

X.drop(['shot_made_flag','shot_id'], axis=1, inplace=True)
X_val.drop(['shot_made_flag','shot_id'], axis=1, inplace=True)
data_submit.drop(['shot_made_flag'], axis=1, inplace=True)


In [26]:
from sklearn import mixture

numGaussians = 13
gaussianMixtureModel = mixture.GMM(n_components=numGaussians, covariance_type='full', 
                                   params='wmc', init_params='wmc',
                                   random_state=1, n_init=3,  verbose=0)
gaussianMixtureModel.fit(X.ix[:,['loc_x','loc_y']])

X['shotLocationCluster'] = gaussianMixtureModel.predict(X.ix[:,['loc_x','loc_y']])



In [ ]:
X['shotLocationCluster']

In [19]:
#data_submit['shotLocationCluster'] = gaussianMixtureModel.predict(data_submit.ix[:,['loc_x','loc_y']])
data_submit = data[mask]
data_submit.drop(['shot_made_flag'], axis=1, inplace=True)

preds = mainLearner.predict_proba(data_submit)[:,1]



submission = pd.DataFrame({'shot_id': shot_id,
                           'shot_made_flag': preds})
submission[['shot_id', 'shot_made_flag']].to_csv('final.csv', index=False)

In [18]:
from sklearn import cross_validation, linear_model, metrics,    preprocessing, ensemble, svm, neighbors
randomSeed = 1
numFolds = 4

mainLearner = ensemble.ExtraTreesClassifier(n_estimators=500, max_depth=5, 
                                            min_samples_leaf=120, max_features=60, 
                                            criterion='entropy', bootstrap=False, 
                                            n_jobs=-1, random_state=randomSeed)

X_val['shotLocationCluster'] = gaussianMixtureModel.predict(X_val.ix[:,['loc_x','loc_y']])

    
    # train learner
mainLearner.fit(X, Y)
    
    # make predictions
pred_val = mainLearner.predict_proba(X_val)[:,1]
import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll
l1 = logloss(Y_val,pred_val)
print l1
   
    


0.617466597267


# Feature Selection

## Univariate Feature Selection: select the top 20 features using chi2 test

In [ ]:
X_minmax = MinMaxScaler(feature_range=(0,1)).fit_transform(X)
X_scored = SelectKBest(score_func=chi2, k='all').fit(X_minmax, Y)
feature_scoring = pd.DataFrame({
        'feature': X.columns,
        'score': X_scored.scores_
    })

feat_scored_20 = feature_scoring.sort_values('score', ascending=False).head(5)['feature'].values
feat_scored_20

## Random Forest: select the 20 most important features

In [ ]:
model = RandomForestClassifier()
model.fit(X, Y)

feature_imp = pd.DataFrame(model.feature_importances_, index=X.columns, columns=["importance"])
feat_imp_20 = feature_imp.sort_values("importance", ascending=False).head(5).index
feat_imp_20

## Recursive Feature Elimination

In [ ]:
rfe = RFE(LogisticRegression(), 5)
rfe.fit(X, Y)

feature_rfe_scoring = pd.DataFrame({
        'feature': X.columns,
        'score': rfe.ranking_
    })

feat_rfe_20 = feature_rfe_scoring[feature_rfe_scoring['score'] == 1]['feature'].values
feat_rfe_20

## Variance Threshold: find all features with more than 90% variance in values

In [ ]:
threshold = 0.9
vt = VarianceThreshold().fit(X)

# Find feature names
feat_var_threshold = X.columns[vt.variances_ > threshold * (1-threshold)]
feat_var_threshold

## Join all the selected features by all methods

In [ ]:
features = np.hstack([
        #feat_var_threshold, 
        feat_imp_20,
        #feat_scored_20,
        #feat_rfe_20
    ])

features = np.unique(features)
print('Final features set:\n')
for f in features:
    print("\t-{}".format(f))

In [ ]:
#data_cl = data_cl.ix[:, features]
data_submit = data_submit.ix[:, features]
X_val = X_val.ix[:,features]
X = X.ix[:, features]




In [ ]:

#print('Clean dataset shape: {}'.format(data_cl.shape))
print('Subbmitable dataset shape: {}'.format(data_submit.shape))
print('Train features shape: {}'.format(X.shape))


In [ ]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
clf = linear_model.LogisticRegression(C=1e5,penalty='l1',max_iter=500)
clf = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=1, random_state=0)
clf.fit(X, Y)

preds = clf.predict(X_val)

import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll
l1 = logloss(Y_val,preds)
print l1

# Classification

In [ ]:
seed = 7
processors=1
num_folds=3
num_instances=len(X)
scoring='log_loss'
kfold = KFold(n=num_instances, n_folds=num_folds, random_state=seed)

In [ ]:
lr_grid = GridSearchCV(
    estimator = LogisticRegression(random_state=seed),
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.001, 0.01, 1, 10, 100, 1000]
    }, 
    cv = kfold, 
    scoring = scoring, 
    n_jobs = processors)

lr_grid.fit(X, Y)

print(lr_grid.best_score_)
print(lr_grid.best_params_)



In [ ]:
clf = RandomForestClassifier(n_jobs=-1, n_estimators=500) 
clf.fit(X,Y)
preds = clf.predict(X_val)



import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

ll = logloss(Y_val,preds)
print ll


## Gradient Boosting Classifier

In [ ]:
clf_GB = GradientBoostingClassifier(loss='deviance', learning_rate=0.012,
                           n_estimators=1000, subsample=0.62, 
                           min_samples_split=2, 
                           min_samples_leaf=0.6, 
                           min_weight_fraction_leaf=0.0, 
                           max_depth=7, 
                           init=None, 
                           random_state=1, 
                           max_features=None, 
                           verbose=0, 
                           max_leaf_nodes=None,
                           warm_start=False, 
                           presort='auto')

clf_GB.fit(X,Y)






In [ ]:
clf_LR = LogisticRegression(solver="newton-cg")
clf_LR.fit(X,Y)



preds = clf_LR.predict(X_val)



import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

ll = logloss(Y_val,preds)
print ll

In [ ]:


submission = pd.DataFrame({'shot_id': shot_id,
                           'shot_made_flag': preds})
submission[['shot_id', 'shot_made_flag']].to_csv('submissionBoosting1.csv', index=False)

In [ ]:
lda_grid = GridSearchCV(
    estimator = LinearDiscriminantAnalysis(),
    param_grid = {
        'solver': ['lsqr'],
        'shrinkage': [0, 0.25, 0.5, 0.75, 1],
        'n_components': [None, 2, 5, 10]
    }, 
    cv = kfold, 
    scoring = scoring, 
    n_jobs = processors)

lda_grid.fit(X, Y)

print(lda_grid.best_score_)
print(lda_grid.best_params_)




In [ ]:
estimator = LinearDiscriminantAnalysis(shrinkage= 0, n_components= None, solver='lsqr')
estimator.fit(X,Y)
preds = estimator.predict(X_val)


import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

ll = logloss(Y_val,preds)
print ll

## Logistic Regression

In [ ]:
seed = 7
processors=1
num_folds=3
num_instances=len(X)
scoring='log_loss'
kfold = KFold(n=num_instances, n_folds=num_folds, random_state=seed)

In [ ]:
from sklearn import cross_validation, linear_model, metrics,    preprocessing, ensemble, svm, neighbors
regressor = linear_model.LogisticRegression(C = 100,random_state = seed, penalty = 'l2')
regressor.fit(X, Y)
acc_val = regressor.score(X_val,Y_val)
print acc_val*100
#preds = regressor.predict(data_submit)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
clf_LDA = LDA()
clf_LDA.fit(X,Y)# <FILL IN> 
acc_test_LDA=clf_LDA.score(X_val,Y_val)# <FILL IN> 


print("The test accuracy of LDA is %2.2f" %(100*acc_test_LDA))

In [ ]:
print data_submit.shape
preds = clf_LDA.predict(data_submit)

submission = pd.DataFrame({'shot_id': shot_id,
                           'shot_made_flag': preds})
submission[['shot_id', 'shot_made_flag']].to_csv('submissionLDASabado.csv', index=False)

In [ ]:
from sklearn import cross_validation, linear_model, metrics,    preprocessing, ensemble, svm, neighbors

clf = linear_model.SGDClassifier(loss='log',penalty = 'l2', n_iter=100)
clf.fit(X, Y)
acc_val = clf.score(X_val,Y_val)
print acc_val * 100

In [ ]:
np.random.seed(0)
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

base_learner = linear_model.SGDClassifier(loss='log',penalty = 'l2', n_iter=100)


        
# Train a real Adaboost classifier and obtain its accuracy
AB_R = AdaBoostClassifier(base_learner, n_estimators=20)
AB_R.fit(X, Y)
acc_AB_R =AB_R.score(X_val, Y_val)

#preds = AB_R.predict(data_submit)

print('Accuracy of real adaboost ensemble is %2.2f '%(100*acc_AB_R))

In [ ]:
clf = svm.SVC(kernel = 'rbf', C = 3)
clf.fit(X, Y)
print clf.score(X_val,Y_val)

## Bagging Classifier

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
base_learner = tree.DecisionTreeClassifier(max_depth=1)
base_learner = linear_model.SGDClassifier(loss='log',penalty = 'l2', n_iter=100)

bagging = BaggingClassifier(base_learner,n_estimators=100)
bagging.fit(X, Y)
acc_test = bagging.score(X_val,Y_val)

print('Accuracy of bagged ensemble is %2.2f '%(100*acc_test))

In [ ]:
preds = bagging.predict(data_submit)
submission = pd.DataFrame({'shot_id': shot_id,
                           'shot_made_flag': preds})
submission[['shot_id', 'shot_made_flag']].to_csv('submissionBAGGING.csv', index=False)



In [ ]:
np.random.seed(0)

from sklearn.ensemble import AdaBoostClassifier

base_learner = linear_model.SGDClassifier(loss='log',penalty = 'l2', n_iter=100)

# Train a discrete Adaboost classifier and obtain its accuracy
AB_D = AdaBoostClassifier(base_learner, n_estimators=80, algorithm='SAMME')
AB_D.fit(X, Y)
acc_AB_D = AB_D.score(X_val, Y_val)
        
# Train a real Adaboost classifier and obtain its accuracy
AB_R = AdaBoostClassifier(base_learner, n_estimators=80, algorithm='SAMME.R')
AB_R.fit(X, Y)
acc_AB_R =AB_R.score(X_val, Y_val)

print('Accuracy of discrete adaboost ensemble is %2.2f '%(100*acc_AB_D))
print('Accuracy of real adaboost ensemble is %2.2f '%(100*acc_AB_R))

In [ ]:
preds = AB_R.predict(X_val)


import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

ll = logloss(Y_val,preds)
print ll